## Mckinsey Hiring Challenge 
### Private Leaderboard rank: 13 (top 6%)
#### Solution

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
import xgboost as xgb

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing
# get bnp & test csv files as a DataFrame
train_df   = pd.read_csv('/home/rishikesh/Dev/Python/Mckinsey/train.csv')
test_df  = pd.read_csv('/home/rishikesh/Dev/Python/Mckinsey/test.csv')

In [7]:
# preview the data
train_df.head()
test_df["Email_ID"].head()
test_df["Email_ID"].head()
	

mctrain=test_df


for feat in train_df.columns:
    if train_df[feat].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(np.unique(list(train_df[feat].values) + list(test_df[feat].values)))
        train_df[feat]   = lbl.transform(list(train_df[feat].values))
        test_df[feat]  = lbl.transform(list(test_df[feat].values))

# remove constant columns
remove = []
for col in train_df.columns:
    if train_df[col].std() == 0:
        remove.append(col)

train_df.drop(remove, axis=1, inplace=True)
test_df.drop(remove, axis=1, inplace=True)

# remove duplicated columns
remove = []
c = train_df.columns
for i in range(len(c)-1):
    v = train_df[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,train_df[c[j]].values):
            remove.append(c[j])

train_df.drop(remove, axis=1, inplace=True)
test_df.drop(remove, axis=1, inplace=True)


for feat in train_df.columns:
    if train_df[feat].dtype == 'float64':
        train_df[feat][np.isnan(train_df[feat])] = train_df[feat].mean()
        test_df[feat][np.isnan(test_df[feat])] = test_df[feat].mean()
      
    elif train_df[feat].dtype == 'object':
        train_df[feat][train_df[feat] != train_df[feat]] = train_df[feat].value_counts().index[0]
        test_df[feat][test_df[feat] != test_df[feat]] = test_df[feat].value_counts().index[0]


/home/rishikesh/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/rishikesh/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
X_train = train_df.drop(["Email_Status","Email_ID"],axis=1)
Y_train = train_df["Email_Status"]
X_test  = test_df.drop("Email_ID",axis=1).copy()

xgtrain = xgb.DMatrix(X_train,Y_train)
xgtest = xgb.DMatrix(X_test)

# preview the data
train_df.head()


,Email_ID,Email_Type,Subject_Hotness_Score,Email_Source_Type,Customer_Location,Email_Campaign_Type,Total_Past_Communications,Time_Email_sent_Category,Word_Count,Total_Links,Total_Images,Email_Status
0,0,1,2.2,2,4,2,33.0,1,440,8.0,0.0,0
1,1,2,2.1,1,8,2,15.0,2,504,5.0,0.0,0
2,2,2,0.1,1,1,3,36.0,2,962,5.0,0.0,1
3,3,1,3.0,2,4,2,25.0,2,610,16.0,0.0,0
4,4,1,0.0,2,2,3,18.0,2,947,4.0,0.0,0


In [9]:
xgboost_params = { "objective":"multi:softmax",    # binary classification 
              "num_class" :3,    # number of classes 
              "eval_metric" : "merror",    # evaluation metric 
              "nthread" : 4,   # number of threads to be used 
              "max_depth": 5,    # maximum depth of tree 
              "eta" : 0.15
                  }
                  
num_round = 111               
                  


In [10]:
best = xgb.cv(xgboost_params, xgtrain, num_round,nfold=5)    

best["test-merror-mean"].min()         
# get prediction

xgtest.num_row()
                  

bst = xgb.train(xgboost_params, xgtrain, num_round);


In [12]:
# get prediction
pred = bst.predict(xgtest);
xgtest.num_row()

test_df  = pd.read_csv('/home/rishikesh/Dev/Python/Mckinsey/test.csv')
submission = pd.DataFrame()
submission["Email_ID"] = test_df["Email_ID"]
submission["Email_Status"] = pred

submission.to_csv('mckinsey.csv', index=False)

In [15]:
X_train.to_csv('xtrain.csv', index=False)

In [16]:
X_test.to_csv('xtest.csv', index=False)

In [17]:
X_train.corr()

,Email_Type,Subject_Hotness_Score,Email_Source_Type,Customer_Location,Email_Campaign_Type,Total_Past_Communications,Time_Email_sent_Category,Word_Count,Total_Links,Total_Images
Email_Type,1.000000,-0.235480,-0.275201,-0.003007,0.233565,-0.044495,-0.000581,0.077938,-0.159864,-0.107505
Subject_Hotness_Score,-0.235480,1.000000,0.006142,-0.006856,-0.550676,-0.419320,-0.003665,-0.237531,-0.027785,-0.020336
Email_Source_Type,-0.275201,0.006142,1.000000,-0.000620,0.035797,-0.066282,0.002137,0.048318,0.056770,0.038621
Customer_Location,-0.003007,-0.006856,-0.000620,1.000000,-0.001422,0.005786,0.002585,-0.003372,-0.000320,-0.000642
Email_Campaign_Type,0.233565,-0.550676,0.035797,-0.001422,1.000000,0.341066,0.001388,0.063540,0.004742,-0.008440
Total_Past_Communications,-0.044495,-0.419320,-0.066282,0.005786,0.341066,1.000000,-0.000790,-0.345251,0.106747,0.060909
Time_Email_sent_Category,-0.000581,-0.003665,0.002137,0.002585,0.001388,-0.000790,1.000000,-0.000508,-0.001964,0.000112
Word_Count,0.077938,-0.237531,0.048318,-0.003372,0.063540,-0.345251,-0.000508,1.000000,0.072257,0.050712
Total_Links,-0.159864,-0.027785,0.056770,-0.000320,0.004742,0.106747,-0.001964,0.072257,1.000000,0.756984
Total_Images,-0.107505,-0.020336,0.038621,-0.000642,-0.008440,0.060909,0.000112,0.050712,0.756984,1.000000


In [31]:
from sklearn.cross_validation import StratifiedKFold

trainingData=StratifiedKFold(X_train.index)

/home/rishikesh/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


In [23]:
trainingData1=StratifiedKFold(X_train.index,n_folds=3)

/home/rishikesh/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


In [25]:
type(trainingData1)

sklearn.cross_validation.StratifiedKFold

In [32]:
i=0
xctrain=np.array
for train_index, test_index in trainingData:
    print(train_index.shape)

(0,)
(68353,)
(68353,)


In [36]:
train_df[train_df["Email_Status"].unique()].count()

Email_ID                 68353
Email_Type               68353
Subject_Hotness_Score    68353
dtype: int64

In [37]:
train_df["Email_Status"].value_counts()

0    54941
1    11039
2     2373
Name: Email_Status, dtype: int64

In [ ]:
train_df[].Email_Status.value_counts()

In [68]:
train1=train_df.ix[0:34176,:]

In [69]:
train2=train_df.ix[34177:68353,:]

In [70]:
train1.shape

(34177, 12)

In [71]:
train1.tail()

,Email_ID,Email_Type,Subject_Hotness_Score,Email_Source_Type,Customer_Location,Email_Campaign_Type,Total_Past_Communications,Time_Email_sent_Category,Word_Count,Total_Links,Total_Images,Email_Status
34172,57569,1,0.1,2,6,3,29.00000,3,757,11.0,0.0,0
34173,57570,1,1.4,2,6,2,28.93325,1,721,14.0,11.0,2
34174,57571,1,0.5,2,1,2,33.00000,1,812,15.0,11.0,0
34175,57572,1,0.2,2,4,3,18.00000,3,713,11.0,4.0,0
34176,57574,2,2.1,1,8,2,17.00000,1,542,3.0,0.0,0


In [73]:
train2.head()

,Email_ID,Email_Type,Subject_Hotness_Score,Email_Source_Type,Customer_Location,Email_Campaign_Type,Total_Past_Communications,Time_Email_sent_Category,Word_Count,Total_Links,Total_Images,Email_Status
34177,57575,1,2.6,2,3,2,28.0,2,520,8.0,0.0,0
34178,57577,1,0.3,1,1,3,22.0,1,251,9.0,0.0,0
34179,57578,1,0.4,2,8,2,33.0,2,700,11.0,0.0,0
34180,57579,1,1.6,2,4,2,22.0,2,827,14.0,9.0,0
34181,57580,1,2.2,1,8,2,12.0,2,694,14.0,0.0,0
